In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Normalização (Estandarização)

In [2]:
# SÉRIE DE DADOS, SEGURO AUTO:
srSegAuto = {'Segurado':['Juliano','Maria', 'Paulo','Rodrigo', 'Raquel','Anaximandro'],
     '%_Fipe':[100, 100, 110, 90, 90, 100],
     'Franquia':[1500, 2500, 2200, 1750, 1900, 2800], 
     'Valor':[2500, 2700, 1900, 1500, 2150, 3000]}

In [3]:
# TRANSPÕE PARA PANDAS
dfSegAuto = pd.DataFrame(srSegAuto)

In [4]:
dfSegAuto.head(10)

,Segurado,%_Fipe,Franquia,Valor
0,Juliano,100,1500,2500
1,Maria,100,2500,2700
2,Paulo,110,2200,1900
3,Rodrigo,90,1750,1500
4,Raquel,90,1900,2150
5,Anaximandro,100,2800,3000


In [5]:
# IMPORTANDO A BIBLIOTECA StandardScaler
# INSTANCIANDO UM OBJETO ESTANDARIZADOR
from sklearn.preprocessing import StandardScaler
estandarizador = StandardScaler()

In [6]:
# TREINANDO O ESTANDARIZADOR A PARTIR DOS DADOS DE SEGURO AUTO
estandarizador.fit(dfSegAuto.drop(['Segurado'],axis=1))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
# A PARTIR DO OBJETO TREINADO, VAMOS TRANSFORMAR NOSSOS DADOS DE SEGURO AUTO EM DADOS NORMALIZADOS
valoresEstand = estandarizador.transform(dfSegAuto.drop(['Segurado'],axis=1))

In [8]:
# POR PADRÃO O RESULTADO PROCESSADO (TRANSFORMADO) POR StandardScaler É ENCAPSULADO EM UM OBJETO DO TIPO ARRAY (NUMPY)
# VAMOS CRIAR UM DF QUE A PARTIR DO ARRAY, INCLUINDO OS NOMES ORIGINAIS DAS COLUNAS
dfSegAutoEstand = pd.DataFrame(valoresEstand, columns = dfSegAuto.drop(['Segurado'],axis=1).columns)
dfSegAutoEstand.head(10)

,%_Fipe,Franquia,Valor
0,0.242536,-1.370545,0.414998
1,0.242536,0.882405,0.813397
2,1.697749,0.206520,-0.780197
3,-1.212678,-0.807307,-1.576994
4,-1.212678,-0.469365,-0.282199
5,0.242536,1.558291,1.410995


In [111]:
# PODEMOS ADD NO DF ORIGINAL AS COLUNAS ESTANDARIZADAS....
dfSegAuto[['%_Fipe_Est', 'Franquia_Est', 'Valor_Est']] = dfSegAutoEstand[['%_Fipe', 'Franquia', 'Valor']]
dfSegAuto.head(10)

,Segurado,%_Fipe,Franquia,Valor,%_Fipe_Est,Franquia_Est,Valor_Est
0,Juliano,100,1500,2500,0.242536,-1.370545,0.414998
1,Maria,100,2500,2700,0.242536,0.882405,0.813397
2,Paulo,110,2200,1900,1.697749,0.206520,-0.780197
3,Rodrigo,90,1750,1500,-1.212678,-0.807307,-1.576994
4,Raquel,90,1900,2150,-1.212678,-0.469365,-0.282199
5,Anaximandro,100,2800,3000,0.242536,1.558291,1.410995


### Clusterização sobre dados normalizados
##### Para ilustrar o uso de dados normalizados, vamos clusterizar os dados do exemplo anterior.

In [112]:
# IMPORTANDO O K-MEANS DO SKLEARN
from sklearn.cluster import KMeans

In [113]:
# DEINIMOS EM K A QUANTIDADE DE GRUPOS QUE QUEREMOS "DIVIDIR OS DADOS"
k = 2

# INSTANCIANDO O OBJETO K-MEANS
modeloKM = KMeans(n_clusters = k)

# FITANDO O MODELO USANDO OS DADOS NORMALIZADOS (dfSegAutoEstand)
modeloKM.fit(X = dfSegAutoEstand)

# ADICIONANDO AO DF OS CLUSTERS, A PARTIR DA PROPRIEDADE LABELS DO OBJETO KMEANS modeloKM
dfSegAuto['Cluster'] = pd.DataFrame(modeloKM.labels_).values

In [114]:
# PRONTO, DADOS CLUSTERIZADOS
dfSegAuto

,Segurado,%_Fipe,Franquia,Valor,%_Fipe_Est,Franquia_Est,Valor_Est,Cluster
0,Juliano,100,1500,2500,0.242536,-1.370545,0.414998,1
1,Maria,100,2500,2700,0.242536,0.882405,0.813397,0
2,Paulo,110,2200,1900,1.697749,0.206520,-0.780197,0
3,Rodrigo,90,1750,1500,-1.212678,-0.807307,-1.576994,1
4,Raquel,90,1900,2150,-1.212678,-0.469365,-0.282199,1
5,Anaximandro,100,2800,3000,0.242536,1.558291,1.410995,0


In [115]:
# A NORMALIZAÇÃO A PARTIR DESSE PONTO PODE NÃO TER MAIS UTILIDADE. PODEMOS EXCLUIR...
dfSegAuto.drop(['%_Fipe_Est', 'Franquia_Est', 'Valor_Est'],axis=1, inplace = True)
dfSegAuto.head(10)

,Segurado,%_Fipe,Franquia,Valor,Cluster
0,Juliano,100,1500,2500,1
1,Maria,100,2500,2700,0
2,Paulo,110,2200,1900,0
3,Rodrigo,90,1750,1500,1
4,Raquel,90,1900,2150,1
5,Anaximandro,100,2800,3000,0


In [116]:
# VAMOS VER SE OS CLUSTERS POSSUEM DIFERENÇA
dfSegAuto.groupby('Cluster')[['%_Fipe', 'Franquia', 'Valor']].mean()

,%_Fipe,Franquia,Valor
Cluster,,,
0,103.333333,2500.000000,2533.333333
1,93.333333,1716.666667,2050.000000


# Redução de Dimensionalidade
### Análise de Componentes Principais (PCA)

In [27]:
# VAMOS TESTAR PCA COM UM DATASET DE EXEMPLO DO SKLEARN: WINE
# O DATASET POSSUI UMA SÉRIE DE MÉTRICAS (DIMENSÕES) SOBRE VINHOS E UMA VARIÁVEL DE DESFECHO: TARGET
# TARGET PODE ASSUMIR 3 CATEGORIAS, ONDE CADA UMA CORRESPONDE A UM TIPO DE VINHO, EX.: PINOT, CABERNET OU MERLOT
from sklearn import datasets

# IMPORTANDO WINE PARA A ESTRUTURA NUMPY
wineNp = datasets.load_wine()

# CRIANDO UM DF
features = pd.DataFrame(data=wineNp['data'],columns=wineNp['feature_names'])
wine = features
# ADICIONANDO O TARGET (NUMÉRICO E CATEGÓRICO)
wine['target']=wineNp['target']
wine['targetCat']=wine['target'].map(lambda ind: wineNp['target_names'][ind])

wine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target,targetCat
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0,class_0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0,class_0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0,class_0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0,class_0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0,class_0


#### Normalização

In [28]:
from sklearn.preprocessing import StandardScaler

# INSTANCIANDO UM OBJETO ESTANDARIZADOR
normWineModel = StandardScaler()

# TREINANDO O ESTANDARIZADOR
normWineModel.fit(wine.drop(['target', 'targetCat'],axis=1))

# APLICANDO O ESTANDARIZADOR SOBRE OS DADOS, GUARDANDO RESULTADO NO ARRAY
normWineData = normWineModel.transform(wine.drop(['target', 'targetCat'],axis=1))

# CRIANDO UM DF COM OS DADOS NORMALIZADOS
dfNormWine = pd.DataFrame(normWineData, columns = wine.drop(['target', 'targetCat'],axis=1).columns)

# PRONTO, AGORA TEMOS O DF NORMALIZADO
dfNormWine.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874


#### Aplicação do PCA

In [30]:
# IMPORTANDO A BIBLIOTECA
from sklearn.decomposition import PCA

# CRIANDO O OBJETO PCA COM 5 COMPONENTES
pca = PCA(n_components = 3)

# FITANDO UM MODELO PCA A PARTIR DOS DADOS NORMALIZADOS
pca.fit(dfNormWine)

# APLICANDO OS DADOS AO MODELO GERADO
X_pca = pca.transform(dfNormWine)

# RESULTADO PCA: TÍNHAMOS 30 AGORA TEMOS 3 VARIÁVEIS
pca.components_

array([[ 0.1443294 , -0.24518758, -0.00205106, -0.23932041,  0.14199204,
         0.39466085,  0.4229343 , -0.2985331 ,  0.31342949, -0.0886167 ,
         0.29671456,  0.37616741,  0.28675223],
       [-0.48365155, -0.22493093, -0.31606881,  0.0105905 , -0.299634  ,
        -0.06503951,  0.00335981, -0.02877949, -0.03930172, -0.52999567,
         0.27923515,  0.16449619, -0.36490283],
       [-0.20738262,  0.08901289,  0.6262239 ,  0.61208035,  0.13075693,
         0.14617896,  0.1506819 ,  0.17036816,  0.14945431, -0.13730621,
         0.08522192,  0.16600459, -0.12674592]])

In [34]:
dfWinePCA = pd.DataFrame(data = X_pca, columns = ['PCA_1', 'PCA_2', 'PCA_3'])

In [35]:
dfWinePCA['target'] = wine['targetCat']

In [36]:
dfWinePCA.head()

,PCA_1,PCA_2,PCA_3,target
0,3.316751,-1.443463,-0.165739,class_0
1,2.209465,0.333393,-2.026457,class_0
2,2.516740,-1.031151,0.982819,class_0
3,3.757066,-2.756372,-0.176192,class_0
4,1.008908,-0.869831,2.026688,class_0
